# Curbage Recognition Experiment Statistical Analyses
### Angelique I. Delarazan
#### 01/31/2022
Analyzes the Curbage Recognition Experiment:
1. Performs two-way ANOVAs on target recognition and lure (2 x 2 ANOVA: Domain Type [Narrarive v. Perceptual] x Age [Older v. Younger])
2. Calculates correlations between neuropsychological tests and target recognition & lure performance
3. Split on MoCA: Incorporate a test of cognitive abilities in Analysis #1 above

Note: Figures generated using R Notebook (see curbage_recognition_figures.Rmd)

## Set Up

In [1]:
import os
import csv
from itertools import islice
import pandas as pd
import scipy.stats
from scipy.stats import norm
import math
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels
import numpy as np

In [2]:
base_dir = '/Volumes/GAMMA/curbage_behavioral/experiment1/'
data_dir = os.path.join(base_dir, f'data/recognition/')
output_dir = os.path.join(base_dir, f'analyses/recognition/')

In [27]:
sub_all_group_all_desc_all= pd.read_csv('/Volumes/GAMMA/curbage_recognition/data/sub-all_group-all_task-recognition_desc-all_results_neuropsych.csv')
#sub_all_group_all_desc_all

,subject,target_hit,lure_cr,foil_cr,target_miss,lure_fa,foil_fa,target_hit_rate,lure_cr_rate,foil_cr_rate,...,MoCA_delayed_recall,MoCA_orientation,MoCA_total,MINT_uncued,MINT_semantic,MINT_total,Craft21_delayed_verbatim,Craft21_delayed_paraphrase,Craft21_delayed_total,group_type
0,201,27,26,30,3,4,0,0.900000,0.866667,1.000000,...,5.0,6.0,29.0,29.0,0.0,29.0,19.0,3.0,22.0,older_narrative
1,201,28,14,30,2,16,0,0.933333,0.466667,1.000000,...,5.0,6.0,29.0,29.0,0.0,29.0,19.0,3.0,22.0,older_perceptual
2,202,29,20,30,1,10,0,0.966667,0.666667,1.000000,...,3.0,6.0,24.0,26.0,0.0,26.0,20.0,1.0,21.0,older_narrative
3,202,26,20,30,4,10,0,0.866667,0.666667,1.000000,...,3.0,6.0,24.0,26.0,0.0,26.0,20.0,1.0,21.0,older_perceptual
4,203,24,23,29,6,7,1,0.800000,0.766667,0.966667,...,5.0,6.0,30.0,30.0,0.0,30.0,10.0,3.0,13.0,older_narrative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,318,28,24,30,2,6,0,0.933333,0.800000,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,younger_perceptual
80,319,30,28,30,0,2,0,0.983333,0.933333,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,younger_perceptual
81,320,28,28,30,2,2,0,0.933333,0.933333,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,younger_perceptual
82,321,29,20,30,1,10,0,0.966667,0.666667,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,younger_perceptual


## Perform two-way ANOVA on target recognition (d')
Target recognition was assessed in terms of normalized d’ values (z[Target Hit Rate] – z[Foil False Alarm Rate]) derived from signal detection analysis.

2 x 2 ANOVA: Domain Type (Perceptual v. Narrative) x Age (Younger v. Older Adults)

In [4]:
#perform two-way ANOVA
model = ols('dprime ~ (group) + (type) + (group):(type)', data=sub_all_group_all_desc_all).fit()
sm.stats.anova_lm(model, typ=2)

,sum_sq,df,F,PR(>F)
group,0.886143,1.0,2.992058,0.087530
type,6.333862,1.0,21.386264,0.000014
group:type,0.110127,1.0,0.371844,0.543729
Residual,23.693198,80.0,NaN,NaN


### Perform Tukey's Post-hoc test

In [5]:
post_hoc = statsmodels.stats.multicomp.pairwise_tukeyhsd(sub_all_group_all_desc_all['dprime'], sub_all_group_all_desc_all['group_type'], alpha=0.05)
print(post_hoc)

           Multiple Comparison of Means - Tukey HSD, FWER=0.05            
      group1            group2       meandiff p-adj   lower  upper  reject
--------------------------------------------------------------------------
  older_narrative   older_perceptual   0.4768 0.0288  0.0361 0.9175   True
  older_narrative  younger_narrative    0.133 0.8408 -0.3077 0.5737  False
  older_narrative younger_perceptual   0.7546  0.001  0.3139 1.1953   True
 older_perceptual  younger_narrative  -0.3438 0.1797 -0.7845 0.0969  False
 older_perceptual younger_perceptual   0.2778 0.3552 -0.1629 0.7185  False
younger_narrative younger_perceptual   0.6216 0.0022  0.1809 1.0623   True
--------------------------------------------------------------------------


-----

## Perform two-way ANOVA on lures (LDI)

Response bias was corrected by calculating a lure discrimination index (LDI) for each subject (p[New|Lure] – p[New|Target])

2 x 2 ANOVA: Domain Type (Perceptual v. Narrative) x Age (Younger v. Older Adults)

In [6]:
model = ols('LDI ~ (group) + (type) + (group):(type)', data=sub_all_group_all_desc_all).fit()
sm.stats.anova_lm(model, typ=2)

,sum_sq,df,F,PR(>F)
group,0.187315,1.0,9.438779,0.002903
type,0.012712,1.0,0.640539,0.425886
group:type,0.086786,1.0,4.373125,0.039683
Residual,1.587619,80.0,NaN,NaN


### Perform Tukey's Post-hoc test

In [7]:
post_hoc = statsmodels.stats.multicomp.pairwise_tukeyhsd(sub_all_group_all_desc_all['LDI'], sub_all_group_all_desc_all['group_type'], alpha=0.05)
print(post_hoc)

           Multiple Comparison of Means - Tukey HSD, FWER=0.05            
      group1            group2       meandiff p-adj   lower  upper  reject
--------------------------------------------------------------------------
  older_narrative   older_perceptual  -0.0889 0.1805  -0.203 0.0252  False
  older_narrative  younger_narrative   0.0302 0.8954 -0.0839 0.1442  False
  older_narrative younger_perceptual   0.0698 0.3818 -0.0442 0.1839  False
 older_perceptual  younger_narrative    0.119 0.0374   0.005 0.2331   True
 older_perceptual younger_perceptual   0.1587 0.0026  0.0447 0.2728   True
younger_narrative younger_perceptual   0.0397 0.7736 -0.0744 0.1538  False
--------------------------------------------------------------------------


-----------

## Calculate mean and standard deviations of all the neuropsychological measures

In [37]:
# Filter current dataframe and remove younger adults
sub_all_group_older_desc_neuropsych = sub_all_group_all_desc_all[sub_all_group_all_desc_all['group'] == 'older']
sub_all_group_older_desc_neuropsych.columns

Index(['subject', 'target_hit', 'lure_cr', 'foil_cr', 'target_miss', 'lure_fa',
       'foil_fa', 'target_hit_rate', 'lure_cr_rate', 'foil_cr_rate',
       'lure_fa_rate', 'foil_fa_rate', 'cr', 'cr_rate', 'fa', 'fa_rate',
       'dprime', 'dprime_lures', 'LDI', 'beta', 'c', 'ad', 'group', 'type',
       'Craft21_immediate_verbatim', 'Craft21_immediate_paraphase',
       'Craft21_immediate_total', 'MoCA_visuospatial/executive', 'MoCA_naming',
       'MoCA_attention', 'MoCA_language', 'MoCA_abstraction',
       'MoCA_delayed_recall', 'MoCA_orientation', 'MoCA_total', 'MINT_uncued',
       'MINT_semantic', 'MINT_total', 'Craft21_delayed_verbatim',
       'Craft21_delayed_paraphrase', 'Craft21_delayed_total', 'group_type'],
      dtype='object')

### MoCA

In [34]:
mean = np.mean(sub_all_group_older_desc_neuropsych['MoCA_total'])
std = scipy.stats.tstd(sub_all_group_older_desc_neuropsych['MoCA_total'])
print(mean, std)

26.047619047619047 3.608610170623973


### Craft 21 Immediate

In [38]:
mean = np.mean(sub_all_group_older_desc_neuropsych['Craft21_immediate_total'])
std = scipy.stats.tstd(sub_all_group_older_desc_neuropsych['Craft21_immediate_total'])
print(mean, std)

20.476190476190474 6.652482006054785


### Craft 21 Delayed

In [39]:
mean = np.mean(sub_all_group_older_desc_neuropsych['Craft21_delayed_total'])
std = scipy.stats.tstd(sub_all_group_older_desc_neuropsych['Craft21_delayed_total'])
print(mean, std)

18.523809523809526 5.49606553391404


### MINT (uncued)

In [40]:
mean = np.mean(sub_all_group_older_desc_neuropsych['MINT_uncued'])
std = scipy.stats.tstd(sub_all_group_older_desc_neuropsych['MINT_uncued'])
print(mean, std)

29.857142857142858 1.932664388255411


### MINT (semantic)

In [41]:
mean = np.mean(sub_all_group_older_desc_neuropsych['MINT_semantic'])
std = scipy.stats.tstd(sub_all_group_older_desc_neuropsych['MINT_semantic'])
print(mean, std)

0.23809523809523808 0.53234519378184


### MINT (total)

In [42]:
mean = np.mean(sub_all_group_older_desc_neuropsych['MINT_total'])
std = scipy.stats.tstd(sub_all_group_older_desc_neuropsych['MINT_total'])
print(mean, std)

30.095238095238095 1.7364039629117973


## Calculate correlations of Narrative and Perceptual task with Neuropsychological Tests

In [8]:
# Filter current dataframe and remove younger adults
sub_all_group_older_desc_neuropsych = sub_all_group_all_desc_all[sub_all_group_all_desc_all['group'] == 'older']
#sub_all_group_older_desc_neuropsych

### MoCA and LDI

In [60]:
r, p  = scipy.stats.pearsonr(sub_all_group_older_desc_neuropsych.query('type=="narrative"')['MoCA_total'], sub_all_group_older_desc_neuropsych.query('type=="narrative"')['LDI'])
print(r, p)

0.04273096590121617 0.8540834995960161


In [61]:
r, p = scipy.stats.pearsonr(sub_all_group_older_desc_neuropsych.query('type=="perceptual"')['MoCA_total'], sub_all_group_older_desc_neuropsych.query('type=="perceptual"')['LDI'])
print(r, p)

0.37392866393574337 0.0949508777060857


### MoCA and d'

In [62]:
r, p  = scipy.stats.pearsonr(sub_all_group_older_desc_neuropsych.query('type=="narrative"')['MoCA_total'], sub_all_group_older_desc_neuropsych.query('type=="narrative"')['dprime'])
print(r, p)

-0.12846077947838547 0.5789341103842007


In [63]:
r, p = scipy.stats.pearsonr(sub_all_group_older_desc_neuropsych.query('type=="perceptual"')['MoCA_total'], sub_all_group_older_desc_neuropsych.query('type=="perceptual"')['dprime'])
print(r, p)

0.02655703690022254 0.9090259823983964


### MINT (uncued) and LDI

In [64]:
r, p  = scipy.stats.pearsonr(sub_all_group_older_desc_neuropsych.query('type=="narrative"')['MINT_uncued'], sub_all_group_older_desc_neuropsych.query('type=="narrative"')['LDI'])
print(r, p)

0.26633989030864147 0.24320383307651267


In [65]:
r, p = scipy.stats.pearsonr(sub_all_group_older_desc_neuropsych.query('type=="perceptual"')['MINT_uncued'], sub_all_group_older_desc_neuropsych.query('type=="perceptual"')['LDI'])
print(r, p)

0.21289006398948085 0.35417071856027943


### MINT (uncued) and d'

In [66]:
r, p  = scipy.stats.pearsonr(sub_all_group_older_desc_neuropsych.query('type=="narrative"')['MINT_uncued'], sub_all_group_older_desc_neuropsych.query('type=="narrative"')['dprime'])
print(r, p)

0.16110057575042097 0.4854100009514949


In [67]:
r, p = scipy.stats.pearsonr(sub_all_group_older_desc_neuropsych.query('type=="perceptual"')['MINT_uncued'], sub_all_group_older_desc_neuropsych.query('type=="perceptual"')['dprime'])
print(r, p)

-0.10680179654040117 0.6449564878807096


### MINT (semantic) and LDI

In [68]:
r, p  = scipy.stats.pearsonr(sub_all_group_older_desc_neuropsych.query('type=="narrative"')['MINT_semantic'], sub_all_group_older_desc_neuropsych.query('type=="narrative"')['LDI'])
print(r, p)

-0.2770208071925857 0.2240977822995993


In [69]:
r, p = scipy.stats.pearsonr(sub_all_group_older_desc_neuropsych.query('type=="perceptual"')['MINT_semantic'], sub_all_group_older_desc_neuropsych.query('type=="perceptual"')['LDI'])
print(r, p)

-0.3520698066875316 0.11753747490161148


### MINT (semantic) and d'

In [70]:
r, p  = scipy.stats.pearsonr(sub_all_group_older_desc_neuropsych.query('type=="narrative"')['MINT_semantic'], sub_all_group_older_desc_neuropsych.query('type=="narrative"')['dprime'])
print(r, p)

-0.19616539025957694 0.39409043574760794


In [71]:
r, p = scipy.stats.pearsonr(sub_all_group_older_desc_neuropsych.query('type=="perceptual"')['MINT_semantic'], sub_all_group_older_desc_neuropsych.query('type=="perceptual"')['dprime'])
print(r, p)

-0.18086083841246886 0.4327110255638251


### MINT (total) and LDI

In [72]:
r, p  = scipy.stats.pearsonr(sub_all_group_older_desc_neuropsych.query('type=="narrative"')['MINT_total'], sub_all_group_older_desc_neuropsych.query('type=="narrative"')['LDI'])
print(r, p)

0.21151467845589172 0.35736183499953356


In [73]:
r, p = scipy.stats.pearsonr(sub_all_group_older_desc_neuropsych.query('type=="perceptual"')['MINT_total'], sub_all_group_older_desc_neuropsych.query('type=="perceptual"')['LDI'])
print(r, p)

0.12901512586069566 0.5772833174813561


### MINT (total) and d'

In [74]:
r, p  = scipy.stats.pearsonr(sub_all_group_older_desc_neuropsych.query('type=="narrative"')['MINT_total'], sub_all_group_older_desc_neuropsych.query('type=="narrative"')['dprime'])
print(r, p)

0.11916906860904693 0.6069014142836762


In [75]:
r, p = scipy.stats.pearsonr(sub_all_group_older_desc_neuropsych.query('type=="perceptual"')['MINT_total'], sub_all_group_older_desc_neuropsych.query('type=="perceptual"')['dprime'])
print(r, p)

-0.1743214328651821 0.44980472460920934


### Craft21 (Immediate) and LDI

In [76]:
r, p  = scipy.stats.pearsonr(sub_all_group_older_desc_neuropsych.query('type=="narrative"')['Craft21_immediate_total'], sub_all_group_older_desc_neuropsych.query('type=="narrative"')['LDI'])
print(r, p)

0.49552974696350727 0.022355932483808058


In [77]:
r, p = scipy.stats.pearsonr(sub_all_group_older_desc_neuropsych.query('type=="perceptual"')['Craft21_immediate_total'], sub_all_group_older_desc_neuropsych.query('type=="perceptual"')['LDI'])
print(r, p)

0.11251200508321545 0.627272071146136


### Craft21 (Delayed) and LDI

In [78]:
r, p  = scipy.stats.pearsonr(sub_all_group_older_desc_neuropsych.query('type=="narrative"')['Craft21_delayed_total'], sub_all_group_older_desc_neuropsych.query('type=="narrative"')['LDI'])
print(r, p)

0.5464351709499262 0.010377536172562707


In [79]:
r, p = scipy.stats.pearsonr(sub_all_group_older_desc_neuropsych.query('type=="perceptual"')['Craft21_delayed_total'], sub_all_group_older_desc_neuropsych.query('type=="perceptual"')['LDI'])
print(r, p)

0.2925681790033601 0.19809399107188994


### Craft21 (Immediate) and d'

In [80]:
r, p  = scipy.stats.pearsonr(sub_all_group_older_desc_neuropsych.query('type=="narrative"')['Craft21_immediate_total'], sub_all_group_older_desc_neuropsych.query('type=="narrative"')['dprime'])
print(r, p)

0.3395930303128523 0.13204344715394678


In [81]:
r, p = scipy.stats.pearsonr(sub_all_group_older_desc_neuropsych.query('type=="perceptual"')['Craft21_immediate_total'], sub_all_group_older_desc_neuropsych.query('type=="perceptual"')['dprime'])
print(r, p)

0.38031875252639097 0.08899673595245042


### Craft21 (Delayed) and d'

In [82]:
r, p  = scipy.stats.pearsonr(sub_all_group_older_desc_neuropsych.query('type=="narrative"')['Craft21_delayed_total'], sub_all_group_older_desc_neuropsych.query('type=="narrative"')['dprime'])
print(r, p)

0.4615712255076184 0.03518076073368875


In [83]:
r, p = scipy.stats.pearsonr(sub_all_group_older_desc_neuropsych.query('type=="perceptual"')['Craft21_delayed_total'], sub_all_group_older_desc_neuropsych.query('type=="perceptual"')['dprime'])
print(r, p)

0.4583208169382502 0.03666032641129591


-----

## Split on MoCA
MoCA scores range from 0-30, and a score of 26 or higher is generally considered as “normal.” Given that our older adult sample exhibited heterogeneity in falling above or below a MoCA score of 26, we incorporated a simple contrast of cognitive ability in our analyses via split based on MoCA score. Older participants were categorized in two groups: Lower MoCA Scores (MoCA score < 26) and Higher MoCA Scores (MoCA score >= 26).

In [25]:
# Generate list of low MoCA scorers
sub_all_group_all_desc_all[sub_all_group_all_desc_all['MoCA_total'] < 26]     

,subject,target_hit,lure_cr,foil_cr,target_miss,lure_fa,foil_fa,target_hit_rate,lure_cr_rate,foil_cr_rate,...,MoCA_delayed_recall,MoCA_orientation,MoCA_total,MINT_uncued,MINT_semantic,MINT_total,Craft21_delayed_verbatim,Craft21_delayed_paraphrase,Craft21_delayed_total,group_type
2,202,29,20,30,1,10,0,0.966667,0.666667,1.000000,...,3.0,6.0,24.0,26.0,0.0,26.0,20.0,1.0,21.0,older_narrative
3,202,26,20,30,4,10,0,0.866667,0.666667,1.000000,...,3.0,6.0,24.0,26.0,0.0,26.0,20.0,1.0,21.0,older_perceptual
12,207,26,24,30,4,6,0,0.866667,0.800000,1.000000,...,3.0,6.0,19.0,31.0,0.0,31.0,19.0,1.0,20.0,older_narrative
13,207,30,20,30,0,10,0,0.983333,0.666667,1.000000,...,3.0,6.0,19.0,31.0,0.0,31.0,19.0,1.0,20.0,older_perceptual
22,212,29,26,30,1,4,0,0.966667,0.866667,1.000000,...,1.0,6.0,18.0,30.0,0.0,30.0,17.0,1.0,18.0,older_narrative
23,212,30,15,30,0,15,0,0.983333,0.500000,1.000000,...,1.0,6.0,18.0,30.0,0.0,30.0,17.0,1.0,18.0,older_perceptual
26,214,27,17,30,3,13,0,0.900000,0.566667,1.000000,...,3.0,6.0,23.0,32.0,0.0,32.0,14.0,2.0,16.0,older_narrative
27,214,29,18,28,1,12,2,0.966667,0.600000,0.933333,...,3.0,6.0,23.0,32.0,0.0,32.0,14.0,2.0,16.0,older_perceptual
28,215,26,25,30,4,5,0,0.866667,0.833333,1.000000,...,2.0,6.0,24.0,31.0,0.0,31.0,16.0,1.0,17.0,older_narrative
29,215,26,20,30,4,10,0,0.866667,0.666667,1.000000,...,2.0,6.0,24.0,31.0,0.0,31.0,16.0,1.0,17.0,older_perceptual


### High MoCA Scorers

In [26]:
# Remove low MoCA scorers
lowMoCA = [202, 207, 214, 212, 215, 217, 220, 221]
sub_highMoCA_group_all_desc_all = sub_all_group_all_desc_all[~sub_all_group_all_desc_all.subject.isin(lowMoCA)]
#sub_highMoCA_group_all_desc_all

### Perform two-way ANOVA on target recognition (d')
Target recognition was assessed in terms of normalized d’ values (z[Target Hit Rate] – z[Foil False Alarm Rate]) derived from signal detection analysis.

2 x 2 ANOVA: Domain Type (Perceptual v. Narrative) x Age (Younger v. Older Adults)

In [52]:
#perform two-way ANOVA
model = ols('dprime ~ (group) + (type) + (group):(type)', data=sub_highMoCA_group_all_desc_all).fit()
sm.stats.anova_lm(model, typ=2)

,sum_sq,df,F,PR(>F)
group,0.604518,1.0,3.053054,8.538224e-02
type,7.464857,1.0,37.700449,5.828014e-08
group:type,0.046262,1.0,0.233642,6.304858e-01
Residual,12.672285,64.0,NaN,NaN


### Perform Tukey's Post-hoc test

In [53]:
post_hoc = statsmodels.stats.multicomp.pairwise_tukeyhsd(sub_highMoCA_group_all_desc_all['dprime'], sub_highMoCA_group_all_desc_all['group_type'], alpha=0.05)
print(post_hoc)

           Multiple Comparison of Means - Tukey HSD, FWER=0.05            
      group1            group2       meandiff p-adj   lower  upper  reject
--------------------------------------------------------------------------
  older_narrative   older_perceptual    0.729  0.001  0.2685 1.1894   True
  older_narrative  younger_narrative   0.2477 0.3999 -0.1665 0.6619  False
  older_narrative younger_perceptual   0.8693  0.001  0.4551 1.2835   True
 older_perceptual  younger_narrative  -0.4813 0.0164 -0.8955 -0.067   True
 older_perceptual younger_perceptual   0.1403 0.7843 -0.2739 0.5546  False
younger_narrative younger_perceptual   0.6216  0.001  0.2594 0.9839   True
--------------------------------------------------------------------------


-----

### Perform two-way ANOVA on lures (LDI)

Response bias was corrected by calculating a lure discrimination index (LDI) for each subject (p[New|Lure] – p[New|Target])

2 x 2 ANOVA: Domain Type (Perceptual v. Narrative) x Age (Younger v. Older Adults)

In [54]:
model = ols('LDI ~ (group) + (type) + (group):(type)', data=sub_highMoCA_group_all_desc_all).fit()
sm.stats.anova_lm(model, typ=2)

,sum_sq,df,F,PR(>F)
group,0.062149,1.0,3.412294,0.069336
type,0.000147,1.0,0.008074,0.928682
group:type,0.037071,1.0,2.035389,0.158535
Residual,1.165649,64.0,NaN,NaN


### Perform Tukey's Post-hoc test

In [56]:
post_hoc = statsmodels.stats.multicomp.pairwise_tukeyhsd(sub_highMoCA_group_all_desc_all['LDI'], sub_highMoCA_group_all_desc_all['group_type'], alpha=0.05)
print(post_hoc)

           Multiple Comparison of Means - Tukey HSD, FWER=0.05            
      group1            group2       meandiff p-adj   lower  upper  reject
--------------------------------------------------------------------------
  older_narrative   older_perceptual  -0.0564  0.689  -0.196 0.0832  False
  older_narrative  younger_narrative   0.0142    0.9 -0.1115 0.1398  False
  older_narrative younger_perceptual   0.0538  0.653 -0.0718 0.1795  False
 older_perceptual  younger_narrative   0.0706 0.4556 -0.0551 0.1962  False
 older_perceptual younger_perceptual   0.1103 0.1054 -0.0154 0.2359  False
younger_narrative younger_perceptual   0.0397 0.7516 -0.0702 0.1495  False
--------------------------------------------------------------------------


### Low MoCA Scorers

In [79]:
sub_lowMoCA_group_all_desc_all = sub_all_group_all_desc_all[sub_all_group_all_desc_all.subject.isin(lowMoCA)]

# Grab all younger adults' data 
sub_all_group_younger_desc_all = sub_all_group_all_desc_all[sub_all_group_all_desc_all.subject > 300]
#sub_all_group_younger_desc_all

# Create dataframe of low MoCA scorers and younger adults
sub_lowMoCA_group_all_desc_all = pd.concat([sub_lowMoCA_group_all_desc_all, sub_all_group_younger_desc_all])
#sub_lowMoCA_group_all_desc_all

In [ ]:
sub_L

### Perform two-way ANOVA on target recognition (d')
Target recognition was assessed in terms of normalized d’ values (z[Target Hit Rate] – z[Foil False Alarm Rate]) derived from signal detection analysis.

2 x 2 ANOVA: Domain Type (Perceptual v. Narrative) x Age (Younger v. Older Adults)

In [74]:
#perform two-way ANOVA
model = ols('dprime ~ (group) + (type) + (group):(type)', data=sub_lowMoCA_group_all_desc_all).fit()
sm.stats.anova_lm(model, typ=2)

,sum_sq,df,F,PR(>F)
group,0.581055,1.0,2.204351,0.143436
type,3.184124,1.0,12.079616,0.001014
group:type,0.891002,1.0,3.380195,0.071486
Residual,14.234117,54.0,NaN,NaN


### Perform Tukey's Post-hoc test

In [75]:
post_hoc = statsmodels.stats.multicomp.pairwise_tukeyhsd(sub_lowMoCA_group_all_desc_all['dprime'], sub_lowMoCA_group_all_desc_all['group_type'], alpha=0.05)
print(post_hoc)

           Multiple Comparison of Means - Tukey HSD, FWER=0.05            
      group1            group2       meandiff p-adj   lower  upper  reject
--------------------------------------------------------------------------
  older_narrative   older_perceptual    0.067    0.9 -0.6135 0.7475  False
  older_narrative  younger_narrative  -0.0534    0.9 -0.6189 0.5121  False
  older_narrative younger_perceptual   0.5682 0.0485  0.0028 1.1337   True
 older_perceptual  younger_narrative  -0.1204    0.9 -0.6858 0.4451  False
 older_perceptual younger_perceptual   0.5013 0.0995 -0.0642 1.0667  False
younger_narrative younger_perceptual   0.6216 0.0014  0.2016 1.0416   True
--------------------------------------------------------------------------


-----

### Perform two-way ANOVA on lures (LDI)

Response bias was corrected by calculating a lure discrimination index (LDI) for each subject (p[New|Lure] – p[New|Target])

2 x 2 ANOVA: Domain Type (Perceptual v. Narrative) x Age (Younger v. Older Adults)

In [76]:
model = ols('LDI ~ (group) + (type) + (group):(type)', data=sub_lowMoCA_group_all_desc_all).fit()
sm.stats.anova_lm(model, typ=2)

,sum_sq,df,F,PR(>F)
group,0.249772,1.0,17.516775,0.000106
type,0.001552,1.0,0.108824,0.742763
group:type,0.095260,1.0,6.680717,0.012481
Residual,0.769987,54.0,NaN,NaN


### Perform Tukey's Post-hoc test

In [77]:
post_hoc = statsmodels.stats.multicomp.pairwise_tukeyhsd(sub_lowMoCA_group_all_desc_all['LDI'], sub_lowMoCA_group_all_desc_all['group_type'], alpha=0.05)
print(post_hoc)

           Multiple Comparison of Means - Tukey HSD, FWER=0.05            
      group1            group2       meandiff p-adj   lower  upper  reject
--------------------------------------------------------------------------
  older_narrative   older_perceptual  -0.1417 0.0947 -0.2999 0.0166  False
  older_narrative  younger_narrative   0.0562 0.6529 -0.0754 0.1877  False
  older_narrative younger_perceptual   0.0958 0.2273 -0.0357 0.2274  False
 older_perceptual  younger_narrative   0.1978 0.0011  0.0663 0.3293   True
 older_perceptual younger_perceptual   0.2375  0.001   0.106  0.369   True
younger_narrative younger_perceptual   0.0397 0.6834  -0.058 0.1374  False
--------------------------------------------------------------------------
